In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import datetime
import joblib
import json
files = json.load(open('/content/drive/Shareddrives/STUDENT-Capstone SS23/files.json','r'))
pd.options.display.max_rows = 100
pd.options.display.width = 150
RANDOM_SEED = 696


In [39]:
use_cols=['DestAirportSeqID', 'DestStateFips', 'CRSElapsedTime', 'Distance','OriginAirportSeqID',
          'Reporting_Airline_AA', 'Reporting_Airline_AS', 'Reporting_Airline_B6', 'Reporting_Airline_CO', 'Reporting_Airline_DL',
          'Reporting_Airline_F9', 'Reporting_Airline_FL', 'Reporting_Airline_MQ', 'Reporting_Airline_NK', 'Reporting_Airline_UA',
          'Reporting_Airline_US', 'Reporting_Airline_VX', 'mfr_AIRBUS', 'mfr_BOEING', 'engine_type_Reciprocating',
          'engine_type_Turbofan', 'engine_type_Turbojet', 'w_type_9', 'w_type_C', 'w_type_N', 'w_type_V',
          'sky_c_det_9', 'sky_c_det_C', 'sky_c_det_M', 'sky_c_det_W', 'sky_cov_00', 'sky_cov_02', 'sky_cov_04',
          'sky_cov_07', 'sky_cov_08', 'sky_cov_09', 'sky_cov_10', 'sky_obs_tot_cov_00',
          'sky_obs_tot_cov_01', 'sky_obs_tot_cov_02', 'sky_obs_tot_cov_04', 'sky_obs_tot_cov_06', 'sky_obs_tot_cov_07',
          'sky_obs_tot_cov_08', 'sky_obs_tot_cov_09', 'w_type_d_9', 'w_type_d_C', 'w_type_d_N', 'w_type_d_V',
          'sky_c_det_d_9', 'sky_c_det_d_C', 'sky_c_det_d_M','sky_c_det_d_W', 'sky_cov_d_00', 'sky_cov_d_02', 'sky_cov_d_04',
          'sky_cov_d_07', 'sky_cov_d_08', 'sky_cov_d_09',
          'sky_cov_d_10', 'sky_obs_tot_cov_d_00', 'sky_obs_tot_cov_d_01', 'sky_obs_tot_cov_d_02',
          'sky_obs_tot_cov_d_03', 'sky_obs_tot_cov_d_04', 'sky_obs_tot_cov_d_06', 'sky_obs_tot_cov_d_07', 'sky_obs_tot_cov_d_08',
          'sky_obs_tot_cov_d_09', 'CRSDepHour', 'passengers', 'no_engines', 'w_dir_angle',
          'w_speed_rate', 'sky_c_hgt', 'vis_dist', 'tmp_air', 'tmp_dew', 'sea_lvl_p', 'liq_precip_qty', 'liq_precip_dim',
          'liq_precip_cond', 'sky_cov_base_hgt', 'sky_cov_cld', 'sky_sum_cov', 'sky_sum_hgt', 'sky_low_cld_base_hgt',
          'at_pres_altimeter_rate', 'at_pres_stn_rate', 'w_dir_angle_d', 'w_speed_rate_d', 'sky_c_hgt_d', 'vis_dist_d',
          'tmp_air_d', 'tmp_dew_d', 'sea_lvl_p_d', 'liq_precip_qty_d', 'liq_precip_dim_d', 'liq_precip_cond_d',
          'sky_cov_base_hgt_d', 'sky_cov_cld_d', 'sky_sum_cov_d', 'sky_sum_hgt_d', 'sky_low_cld_base_hgt_d',
          'at_pres_altimeter_rate_d', 'at_pres_stn_rate_d']

# removed from use_cols 'Weather_Label', 'Year',

features_to_scale = ['CRSElapsedTime', 'Distance', 'passengers', 'OriginAirportSeqID','DestAirportSeqID',
               'w_dir_angle','w_speed_rate','sky_c_hgt','vis_dist','tmp_air','tmp_dew',
              'sea_lvl_p','liq_precip_qty','liq_precip_dim','sky_cov_base_hgt','sky_cov_cld','sky_sum_cov','sky_sum_hgt',
              'sky_low_cld_base_hgt','at_pres_altimeter_rate','at_pres_stn_rate','w_dir_angle_d',
              'w_speed_rate_d','sky_c_hgt_d','vis_dist_d','tmp_air_d','tmp_dew_d',
              'sea_lvl_p_d','liq_precip_qty_d','liq_precip_dim_d','liq_precip_cond_d','sky_cov_base_hgt_d',
              'sky_cov_cld_d', 'sky_sum_cov_d','sky_sum_hgt_d','sky_low_cld_base_hgt_d','at_pres_altimeter_rate_d',
              'at_pres_stn_rate_d']
# removed 'Year' from here as well

Downsampled Modeling

In [48]:
df_dev = pd.read_parquet(files['Final_Sets']['Training']['Train_Down'])
df_train = pd.read_parquet(files['Final_Sets']['Training']['Dev_Down'])
dfw_train=pd.concat([df_dev,df_train])

In [49]:
X=dfw_train[use_cols].copy()
y=dfw_train[['Weather_Label']]

numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Create a ColumnTransformer to apply the transformer to specific columns
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, features_to_scale)
    ],verbose_feature_names_out=False).set_output(transform='pandas')

# Fit and transform the data using the preprocessor
X[features_to_scale] = preprocessor.fit_transform(X[features_to_scale])

In [52]:
lr_model = LogisticRegression(random_state=RANDOM_SEED, max_iter = 150, C= 0.5, n_jobs=6, class_weight='balanced', solver= 'newton-cg')
lr_model.fit(X, y.values.ravel())

LogisticRegression(C=0.5, class_weight='balanced', max_iter=150, n_jobs=6,
                   random_state=696, solver='newton-cg')

In [55]:
# saving the model
joblib.dump(lr_model, files['Models']['LR'])

['/content/drive/Shareddrives/STUDENT-Capstone SS23/Models/LR_trained_model.pkl']

In [54]:
joblib.dump(preprocessor, files['Models']['pipe'])

['/content/drive/Shareddrives/STUDENT-Capstone SS23/Models/PreProcessor.pkl']